In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
%run ../py/sql_utlis.py

ZMQInteractiveShell_obj = get_ipython()
su = SqlUtilities()
_, CURSOR = su.get_jh_conn_cursor()
# su.create_navigableparentsequence_table(CURSOR)
# su.populate_navigableparentsequence_table(CURSOR)

Wall time: 10 s


In [16]:

sql_str = '''
    SELECT *
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps;'''
nps_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
mask_series = (nps_df.mrs_id != 0)
file_count = nps_df[mask_series].file_name_id.unique().shape[0]
example_count = nps_df[mask_series].shape[0]
print(f' I have {example_count} non-zero examples in {file_count} files')
nps_df[mask_series].sample(5)

 I have 64 non-zero examples in 7 files


,navigable_parent_sequence_id,file_name_id,navigable_parent_id,sequence_order,mrs_id
581,889,13,6676,23,2
7130,10312,180,10781,30,2
6760,9376,171,6969,25,3
575,883,13,5874,17,2
10203,14616,249,2453,41,2


In [19]:

sequence_dict = {}
def f(df):
    file_name_id = df.file_name_id.tolist()[0]
    sequence_dict[file_name_id] = df.set_index('navigable_parent_sequence_id').sort_values('sequence_order')
nps_df.groupby('file_name_id').apply(f)
mask_series = (nps_df.mrs_id != 0)
file_name_ids_list = nps_df[mask_series].file_name_id.unique().tolist()
for file_name_id in file_name_ids_list:
    sequence_dict.pop(file_name_id, None)

In [20]:

# su.create_minimumrequirementssection_table(CURSOR)
# su.populate_minimumrequirementssection_table(CURSOR)
sql_str = '''
    SELECT *
    FROM [Jobhunting].[dbo].[MinimumRequirementsSection] mrs;'''
mrs_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False)).set_index('mrs_id')
mrs_df.mrs_symbol.to_dict()

{0: 'N', 1: 'B', 2: 'M', 3: 'E'}

In [21]:

import random
import textwrap

file_name_id = random.sample(list(sequence_dict), 1)[0]
df = sequence_dict[file_name_id]
navigable_parent_id = random.sample(df.navigable_parent_id.tolist(), 1)[0]
sql_str = f'''
    SELECT np.[navigable_parent]
    FROM [Jobhunting].[dbo].[NavigableParents] np
    WHERE np.[navigable_parent_id] = {navigable_parent_id};'''
html_str = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False)).navigable_parent.squeeze()
print('\n# ' + '\n# '.join(textwrap.wrap(html_str)))


# <b>SUMMARY</b>


In [29]:

def get_sql_code(sequence_dict):
    file_name_id = random.sample(list(sequence_dict), 1)[0]
    df = sequence_dict[file_name_id]
    output_str = ''
    sql_str = '''
        SELECT *
        FROM [Jobhunting].[dbo].[NavigableParentSequence] nps;'''
    nps_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
    mask_series = (nps_df.mrs_id != 0)
    file_count = nps_df[mask_series].file_name_id.unique().shape[0]
    example_count = nps_df[mask_series].shape[0]
    output_str += f'\n# You now have {example_count} non-zero examples in {file_count} files\n'
    for navigable_parent_sequence_id, row_series in df.iterrows():
        navigable_parent_id = row_series.navigable_parent_id
        sql_str = f'''
            SELECT np.[navigable_parent]
            FROM [Jobhunting].[dbo].[NavigableParents] np
            WHERE np.[navigable_parent_id] = {navigable_parent_id};'''
        html_str = pd.DataFrame(su.get_execution_results(CURSOR, sql_str)).navigable_parent.squeeze()
        output_str += '\n# ' + '\n# '.join(textwrap.wrap(html_str))
        output_str += '\n# ' + str(mrs_df.mrs_symbol.to_dict())
        mrs_id = row_series.mrs_id
        if mrs_id == 0:
            mrs_id = 'xx'
        output_str += f'\nsql_str = r"""UPDATE NavigableParentSequence\n    SET\n        mrs_id = {mrs_id}\n'
        output_str += f'    WHERE (navigable_parent_sequence_id = {navigable_parent_sequence_id})"""\n'
        output_str += 'cursor_obj = CURSOR.execute(sql_str)'
        output_str += '\nCURSOR.commit()\n'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_sql_code(sequence_dict), replace=True)